In [ ]:
#This uses geopy36
# import geopandas as gpd
import pandas as pd
%matplotlib inline  
# from shapely.geometry import Point

# from simpledbf import Dbf5
#pip install simpledbf
#https://pypi.python.org/pypi/simpledbf/0.2.4

In [ ]:
years = ['2015','2020','2025','2030','2035','2040']

In [ ]:
for year in years:
    abag = pd.read_csv('ABAG_03202018/run7224c_taz_summaries_' + year + '.csv')
    abag = abag.rename(columns={'ZONE':'TAZ1454',"HHINCQ1":"HH1","HHINCQ2":"HH2","HHINCQ3":"HH3","HHINCQ4":"HH4"})

    for rp in ['bart','ccag']:
        rp_calcs = pd.read_csv('output/' + rp + '_proj_abag_calcs_' + year + '.csv')
        # output/ccag_proj_abag_calcs_' + year + '.csv',index=False)

        rp_calcs_final = rp_calcs[['TAZ', 'DIST', 'SDIST', 'COUNTY', 'abag_TOTHH_dist', 'abag_HHPOP_dist',
               'abag_TOTPOP_dist', 'abag_EMPRES_dist', 'abag_SFHH_dist', 'abag_MFHH_dist', 'abag_HH1_dist', 'abag_HH2_dist', 'abag_HH3_dist', 'abag_HH4_dist',
               'INC1', 'INC2', 'INC3', 'INC4', 'MHHINC', 'TACRES', 'abag_RESACRE_dist',
               'abag_CIACRE_dist', 'Z2SHARE', 'abag_TEMP_dist', 'abag_RETEMP_dist', 'abag_SEREMP_dist', 'abag_OTHEMP_dist', 'abag_AGEMP_dist',
               'abag_MANEMP_dist', 'abag_WHOEMP_dist', 'abag_AGE0004_dist', 'abag_AGE0519_dist', 'abag_AGE2044_dist', 'abag_AGE4564_dist',
               'abag_AGE65_dist', 'abag_AGE0513_dist', 'abag_AGE1417_dist', 'abag_AGE1824_dist', 'ESENR', 'HSENR',
               'COLLENR', 'COLLENRF', 'COLLENRP','TAZ1454']].rename(columns={'TAZ':'ZONE'})

        rp_calcs_final.rename(columns=lambda x: x.replace('abag_','').replace('_dist',''),inplace=True)
        rp_calcs_final.rename(columns={"TEMP":"TOTEMP"}, inplace=True)

    # del ccag_calcs_final['TAZ1454']
        checkfile = abag.sort_values(by='TAZ1454').set_index('COUNTY','TAZ1454') - rp_calcs_final.groupby(['COUNTY','TAZ1454']).sum().reset_index().sort_values(by='TAZ1454').set_index('COUNTY','TAZ1454')
        checkfile.dropna(axis='columns', how='all').to_csv("checks/" + rp + "_" + year + "_relevant_columns_by_county_and_TAZ.csv")

        checkfile = abag.sort_values(by='TAZ1454').set_index('TAZ1454') - rp_calcs_final.groupby(['TAZ1454']).sum().reset_index().sort_values(by='TAZ1454').set_index('TAZ1454')
        checkfile.to_csv("checks/" + rp + "_" + year + "_columns_by_TAZ.csv")
        checkfile.dropna(axis='columns', how='all').to_csv("checks/" + rp + "_" + year + "_relevant_columns_by_TAZ.csv")

        county_checkfile = abag.sort_values(by='TAZ1454').groupby(['COUNTY']).sum() - rp_calcs_final.groupby(['COUNTY']).sum()
        county_checkfile.dropna(axis='columns', how='all').to_csv("checks/" + rp + "_" + year + "_relevant_columns_by_County.csv")